<a href="https://colab.research.google.com/github/salmaasmanadia/bookstore-testing/blob/main/automatic_sertificate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Library & Import Configuration

In [ ]:
!pip install pandas pillow reportlab openpyxl

import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from reportlab.pdfgen import canvas
import os
import smtplib
from email.message import EmailMessage


# Upload File Excel Data Peserta

In [94]:
from google.colab import files

uploaded = files.upload()


Saving data peserta.xlsx to data peserta (1).xlsx


# Upload File Gambar Template Sertifikat

In [93]:
from google.colab import files

uploaded = files.upload()


Saving [TEMPLATE] Master Sertifikat SPM NEW AIS_page-0001 (1).jpg to [TEMPLATE] Master Sertifikat SPM NEW AIS_page-0001 (1) (1).jpg


# Load Data

In [95]:
df = pd.read_excel("data peserta.xlsx", header=0)
df


,nama,email,acara,tanggal,no_sertifikat
0,Salma Asma Nadia,calmiaws@gmail.com,Sharing Session Pasar Modal,27 Januari 2026,IDX-001
1,Nadia Asma Salma,00salmanadia00@gmail.com,Sharing Session Pasar Modal,28 Januari 2026,IDX-002
2,Salma Nadia Nadia Nadia Nadia Nadia,salma.asma.nadia@mail.ugm.ac.id,Sharing Session Pasar Modal,29 Januari 2026,IDX-003


# Generate Sertificate -> PDF & JPG

In [96]:
def generate_certificate_png(data):
    img = Image.open("[TEMPLATE] Master Sertifikat SPM NEW AIS_page-0001 (1).jpg")
    draw = ImageDraw.Draw(img)

    FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"

    nama = data["nama"].upper()

    # AREA MAKSIMAL (SAMPAI SEBELUM LOGO KANAN)
    MAX_WIDTH = img.width * 0.6

    # auto resize biar tidak tembus kanan
    font = get_auto_font(
        draw,
        nama,
        FONT_PATH,
        max_width=MAX_WIDTH,
        start_size=50,
        min_size=28
    )

    # 🎯 ANCHOR KIRI (TETAP)
    x = 180
    y = 700

    draw.text((x, y), nama, fill="black", font=font)

    os.makedirs("output", exist_ok=True)

    img_path = f"output/Sertifikat_{data['nama']}.jpg"
    pdf_path = f"output/Sertifikat_{data['nama']}.pdf"

    img.save(img_path)

    from reportlab.pdfgen import canvas
    c = canvas.Canvas(pdf_path, pagesize=(img.width, img.height))
    c.drawImage(img_path, 0, 0, width=img.width, height=img.height)
    c.save()

    return pdf_path


In [97]:
for _, row in df.iterrows():
    generate_certificate_png(row)

print("✅ Semua sertifikat berhasil dibuat (JPG & PDF)")


✅ Semua sertifikat berhasil dibuat (JPG & PDF)


# Template Body Email

In [98]:
def email_body(nama):
    return f"""
Yth. {nama},

Terima kasih telah mengikuti kegiatan
Sekolah Pasar Modal Bursa Efek Indonesia Kantor Perwakilan Yogyakarta.

Bersama email ini kami lampirkan
e-Sertifikat dalam format PDF
sebagai bukti keikutsertaan Anda.

Apabila terdapat kesalahan penulisan nama,
silakan menghubungi panitia.

Hormat kami,
Panitia Sekolah Pasar Modal
"""

# Email Configuration

In [99]:
EMAIL_PENGIRIM = "beikpyogyakartasertificate@gmail.com"
EMAIL_PASSWORD = "zwcsucfhgdxnstxo"       # app password TANPA spasi, ini bukan password email

# Send Email +  Attachment

In [100]:
# cek dulu emailnya berhasil ke kirim ngga sebelum dikirim massal
send_email(
    to_email="salmasmanadia@gmail.com",
    subject="TEST EMAIL SERTIFIKAT",
    body="Jika email ini masuk, berarti App Password berhasil.",
    attachment_path=pdf_test
)


In [101]:
for _, row in df.iterrows():
    pdf_path = generate_certificate_png(row)
    body = email_body(row["nama"])
    send_email(
        to_email=row["email"],
        subject="E-Sertifikat Sekolah Pasar Modal",
        body=body,
        attachment_path=pdf_path
    )

print("✅ Semua sertifikat berhasil dikirim via email")


✅ Semua sertifikat berhasil dikirim via email
